#### Template notebook for decomposing input video into frames with subsequent frame compression.

In [1]:
library("jpeg")
library("imager")

Warning message:
"package 'imager' was built under R version 3.6.3"
Loading required package: magrittr


Attaching package: 'imager'


The following object is masked from 'package:magrittr':

    add


The following objects are masked from 'package:stats':

    convolve, spectrum


The following object is masked from 'package:graphics':

    frame


The following object is masked from 'package:base':

    save.image




In [3]:
getFileNameAndExtension <- function(file){ 
    ex <- strsplit(basename(file), split="\\.")
    ex<-unlist(ex)
    return(ex)
} 

In [4]:
convertVideoAsImages<-function(ffmpeg,videoFileName,fps,filePrefix,fileSuffix)
{
 nameAndExtension<-getFileNameAndExtension(videoFileName)
 utils::browseURL(videoFileName)
 dir.create(nameAndExtension[[1]])
 command<-sprintf("%s -i %s -vf fps=%d ./%s/%s%%d%s",ffmpeg,videoFileName,fps,nameAndExtension[[1]],filePrefix,fileSuffix)
 command
 system(command)
}

In [ ]:
firstVideoFileName<-'one.mp4'
ffmpeg<-'c:/ffmpeg/bin/ffmpeg'
outFilePrefix='out'
outFileSuffix='.jpg'
fps=10
convertVideoAsImages(ffmpeg,firstVideoFileName,fps,outFilePrefix,outFileSuffix)

In [ ]:
nameAndExtension<-getFileNameAndExtension(firstVideoFileName)
fnames <- paste0(nameAndExtension[[1]],"/",outFilePrefix, 1, outFileSuffix)
jj <- readJPEG(fnames,native=TRUE)
plot(0:1,0:1,type="n",ann=FALSE,axes=FALSE)
rasterImage(jj,0,0,1,1)

In [ ]:
compressRatio=40
nameAndExtension<-getFileNameAndExtension(firstVideoFileName)
fnames <- paste0(nameAndExtension[[1]],"/",outFilePrefix, 100, outFileSuffix)
sampleImage <- load.image(fnames)
plot(sampleImage)

In [ ]:
thmb <- resize(sampleImage,round(width(sampleImage)/compressRatio),round(height(sampleImage)/compressRatio))
gs<-grayscale(thmb)
plot(gs)

In [ ]:
gs.matrix<-as.matrix(gs)
dim(gs.matrix)

In [ ]:
plot(thmb,main="Thumbnail")

In [ ]:
outFileName<-sprintf("%sthmbout1.jpg",nameAndExtension[[1]])
imager::save.image(thmb,outFileName)

In [ ]:
reduceImagesfromFolder<-function(reductionRatio,thumbNailFolderPrefix,imageFolder,prefix)
{
imageNames<-paste0(prefix,"*.*")
files<-list.files(path=imageFolder,pattern=imageNames,all.files=T,full.name=T,no..=T)
list_of_images=lapply(files,load.image)
n.images<-length(list_of_images)
outFolderName<-sprintf("%s%s",thumbNailFolderPrefix,imageFolder)
outFileName<-sprintf("%s%s",thumbNailFolderPrefix,files[[1]])
if(!dir.exists(outFolderName))
{
  dir.create(path=outFolderName)  
}
for(ii in 1:n.images)
{
  outFileName<-sprintf("%s%s",thumbNailFolderPrefix,files[[ii]])
  thmb <- resize(list_of_images[[ii]],round(width(list_of_images[[ii]])/reductionRatio),round(height(list_of_images[[ii]])/reductionRatio))
  thmb<-grayscale(thmb)
  imager::save.image(thmb,outFileName)
}
}

In [ ]:
thumbNailFolder<-"Small"
nameAndExtension<-getFileNameAndExtension(firstVideoFileName)
reduceImagesfromFolder(compressRatio,thumbNailFolder,nameAndExtension[[1]],outFilePrefix)

In [ ]:
nameAndExtension<-getFileNameAndExtension(secondVideoFileName)
reduceImagesfromFolder(compressRatio,thumbNailFolder,nameAndExtension[[1]],outFilePrefix)